In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [6]:
# !{sys.executable} -m pip install dscribe

In [1]:
# !conda install -c conda-forge rdkit -y

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [3]:
from pytorch_geometric2 import *

In [4]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime
import random

# import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# from torch_geometric.data import Data, DataLoader
# import torch
# import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
# from torch_geometric.datasets import QM9
# from torch_geometric.datasets import TUDataset
# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv, ChebConv  # noqa
# from torch.nn import Sequential, Linear, ReLU, GRU
# import torch_geometric.transforms as T
# from torch_geometric.datasets import QM9
# from torch_geometric.nn import NNConv, Set2Set
# from torch_geometric.data import DataLoader
# from torch_geometric.utils import remove_self_loops
# from torch_geometric.data import Data

from torch.utils.data import DataLoader

In [5]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

In [6]:
file_folder = '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [7]:
graph_dir='../../data/temp/pytorch_geometric2'

In [6]:
# run_convert_to_graph(graph_dir='../../data/temp/pytorch_geometric2', csv_dir=file_folder)

In [8]:
train = pd.read_csv(f'{file_folder}/train.csv')

In [9]:
test = pd.read_csv(f'{file_folder}/train.csv')

In [10]:
molecule_names_test = test.molecule_name.unique().tolist()

In [11]:
molecule_names = train.molecule_name.unique().tolist()

In [12]:
from torch_geometric.utils import scatter_
from torch_scatter import *


def train(net, schduler, optimizer, train_loader, valid_loader):
#     start = timer()

#     num_iters   = 3000  *1000
#     train_loss   = np.zeros(20,np.float32)
#     valid_loss   = np.zeros(20,np.float32)
#     batch_loss   = np.zeros(20,np.float32)

#     iter_smooth = 50
#     iter_accum  = 1
#     iter_valid  = 500
#     start_epoch= 0
#     start_iter = 0
#     iter_ = 0
#     i    = 0
    
    
    epochs = 100
    iter_accum=1
    iter_ = 0
    for epoch in range(epochs):
        train_loss = 0
        optimizer.zero_grad()
        for node, edge, edge_index, node_index, coupling_value, coupling_index, infor in train_loader:
            # learning rate schduler -------------
            lr = schduler(iter_)
            if lr<0 : break
            adjust_learning_rate(optimizer, lr)
            rate = get_learning_rate(optimizer)
            
            net.train()
            node = node.cuda()
            edge = edge.cuda()
            edge_index = edge_index.cuda()
            node_index = node_index.cuda()
            coupling_value = coupling_value.cuda()
            coupling_index = coupling_index.cuda()
            predict = net(node, edge, edge_index, node_index, coupling_index)
            loss = criterion(predict, coupling_value)
            
            (loss/iter_accum).backward()
            if (iter_ % iter_accum)==0:
                optimizer.step()
                optimizer.zero_grad()
            train_loss += loss.item()
            
            iter_ += 1
            
        train_loss /= len(infor)
        valid_loss = do_valid(net, valid_loader)
        print(f'epoch {epoch}, train_loss {train_loss}, valid_loss{valid_loss}')


def do_valid(net, valid_loader):

    valid_num = 0
    valid_predict = []
    valid_coupling_type  = []
    valid_coupling_value = []

    valid_loss = 0
    for b, (node, edge, edge_index, node_index, coupling_value, coupling_index, infor) in enumerate(valid_loader):

        #if b==5: break
        net.eval()
        node = node.cuda()
        edge = edge.cuda()
        edge_index = edge_index.cuda()
        node_index = node_index.cuda()

        coupling_value = coupling_value.cuda()
        coupling_index = coupling_index.cuda()

        with torch.no_grad():
            predict = net(node, edge, edge_index, node_index, coupling_index)
            loss = criterion(predict, coupling_value)

        #---
        batch_size = len(infor)
        valid_predict.append(predict.data.cpu().numpy())
        valid_coupling_type.append(coupling_index[:,2].data.cpu().numpy())
        valid_coupling_value.append(coupling_value.data.cpu().numpy())

        valid_loss += batch_size*loss.item()
        valid_num  += batch_size

        print('\r %8d /%8d'%(valid_num, len(valid_loader.dataset)),end='',flush=True)

        pass  #-- end of one data loader --
    print(valid_num,len(valid_loader.dataset) )
#     assert(valid_num == len(valid_loader.dataset))
    #print('')
    valid_loss = valid_loss/valid_num

    #compute
    predict = np.concatenate(valid_predict)
    coupling_value = np.concatenate(valid_coupling_value)
    coupling_type  = np.concatenate(valid_coupling_type).astype(np.int32)
    mae, log_mae   = compute_kaggle_metric( predict, coupling_value, coupling_type,)

    num_target = NUM_COUPLING_TYPE
    for t in range(NUM_COUPLING_TYPE):
        if mae[t] is None:
            mae[t] = 0
            log_mae[t]  = 0
            num_target -= 1

    mae_mean, log_mae_mean = np.sum(mae)/num_target, np.sum(log_mae)/num_target
    #list(np.stack([mae, log_mae]).T.reshape(-1))

    valid_loss = log_mae + [valid_loss,mae_mean, log_mae_mean, ]
    return valid_loss

def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

def criterion(predict, truth):
    predict = predict.view(-1)
    truth   = truth.view(-1)
    assert(predict.shape==truth.shape)

    loss = torch.abs(predict-truth)
    loss = loss.mean()
    loss = torch.log(loss)
    return loss


def criterion_mae(predict, truth):
    predict = predict.view(-1)
    truth   = truth.view(-1)
    assert(predict.shape==truth.shape)

    loss = torch.abs(predict-truth)
    loss = loss.mean()
#     loss = torch.log(loss)
    return loss

In [25]:

batch_size=16
node_dim = 93
edge_dim =  6
num_target = 8


cv = KFold(n_splits= 2, shuffle= True, random_state= 42)
splits = cv.split(molecule_names[:32]) 
for fold_n, (train_index, valid_index) in enumerate(splits):
    
    train_molecule_names = [molecule_names[i] for i in train_index]
    valid_molecule_names = [molecule_names[i] for i in valid_index]
    
#     assert len(train_molecule_names) + len(valid_molecule_names) == len(molecule_names), 'len is not match'
    train_dataset = ChampsDataset(molecule_names=train_molecule_names, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    valid_dataset = ChampsDataset(molecule_names=valid_molecule_names, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, sampler=RandomSampler(valid_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    
    net = Net(node_dim=node_dim,edge_dim=edge_dim, num_target=num_target).cuda()
    schduler = NullScheduler(lr=0.001)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=schduler(0))
    
    train2(net, schduler, optimizer, train_loader, valid_loader)
    
    break

RuntimeError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 7.44 GiB total capacity; 328.14 MiB already allocated; 45.31 MiB free; 7.86 MiB cached)

In [22]:
test_dataset = ChampsDataset(molecule_names=molecule_names_test, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
test_loader = DataLoader(test_dataset, batch_size=1, sampler=RandomSampler(test_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)

In [21]:


valid_num = 0
valid_predict = []
valid_coupling_type  = []
valid_coupling_value = []

valid_loss = 0
df_pred = pd.DataFrame()
for b, (node, edge, edge_index, node_index, coupling_value, coupling_index, infor) in enumerate(test_loader):

    #if b==5: break
    net.eval()
    node = node.cuda()
    edge = edge.cuda()
    edge_index = edge_index.cuda()
    node_index = node_index.cuda()

    coupling_value = coupling_value.cuda()
    coupling_index = coupling_index.cuda()

    with torch.no_grad():
        predict = net(node, edge, edge_index, node_index, coupling_index)
    df_pred_i = pd.DataFrame({'id':infor[0][2], 'scalar_coupling_constant':predict.cpu().detach().numpy() })
    df_pred = pd.concat([df_pred, df_pred_i], axis=0)

NameError: name 'test_loader' is not defined

In [45]:
df_pred.shape

(4658147, 2)

In [46]:
df_pred.to_csv('../../data/submission/submission_gnn.csv', index=False)

In [47]:
torch.save(net.state_dict(),'net')
torch.save({'optimizer': optimizer.state_dict(),'iter'     : 0, 'epoch'    : 0,}, opt)

NameError: name 'opt' is not defined

In [35]:
test[test['molecule_name']=='dsgdb9nsd_072373'].shape

(40, 6)

In [43]:
infor[0][2]

array([2362542, 2362543, 2362544, 2362545, 2362546, 2362547, 2362548,
       2362549, 2362550, 2362551, 2362552, 2362553, 2362554, 2362555,
       2362556, 2362557, 2362558, 2362559, 2362560, 2362561, 2362562,
       2362563, 2362564, 2362565, 2362566, 2362567, 2362568, 2362569,
       2362570, 2362571, 2362572, 2362573, 2362574, 2362575, 2362576,
       2362577, 2362578, 2362579, 2362580, 2362581])

In [33]:
predict.cpu().detach().numpy() 

array([ -1.618536  ,   3.0760603 ,   3.7532988 ,   7.385342  ,
        -4.48687   ,  88.83468   ,  -5.0857706 ,   6.009048  ,
        -0.96180224,   3.0475516 , -16.347216  ,   2.2974215 ,
         0.2072541 ,  85.91742   ,  -3.3111904 ,   5.802936  ,
         4.0546107 ,   7.406941  ,   2.5372365 ,   6.3977003 ,
         7.789091  , 121.347305  ,   2.435028  ,  10.302857  ,
         3.4585402 ,   5.5056796 ,   8.21506   ,   2.007053  ,
       123.080154  ,   6.8563123 ,   1.4708134 ,  -0.48494583,
         1.445515  ,   6.9501786 ,   2.1423066 ,  97.933044  ,
         1.8729662 ,  11.17389   , -12.52721   ,  36.56184   ],
      dtype=float32)

In [ ]:
idx=0.35
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)

In [15]:
class NullScheduler():
    def __init__(self, lr=0.01 ):
        super(NullScheduler, self).__init__()
        self.lr    = lr
        self.cycle = 0

    def __call__(self, time):
        return self.lr

    def __str__(self):
        string = 'NullScheduler\n' \
                + 'lr=%0.5f '%(self.lr)
        return string

class LinearBn(nn.Module):
    def __init__(self, in_channel, out_channel, act=None):
        super(LinearBn, self).__init__()
        self.linear = nn.Linear(in_channel, out_channel, bias=False)
        self.bn   = nn.BatchNorm1d(out_channel,eps=1e-05, momentum=0.1)
        self.act  = act

    def forward(self, x):
        x = self.linear(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.act is not None:
            x = self.act(x)
        return x

class GraphConv(nn.Module):
    def __init__(self, node_dim, edge_dim ):
        super(GraphConv, self).__init__()

        self.encoder = nn.Sequential(
            LinearBn(edge_dim, 256),
            nn.ReLU(inplace=True),
            LinearBn(256, 256),
            nn.ReLU(inplace=True),
            LinearBn(256, 128),
            nn.ReLU(inplace=True),
            LinearBn(128, node_dim * node_dim),
            #nn.ReLU(inplace=True),
        )

        self.gru  = nn.GRU(node_dim, node_dim, batch_first=False, bidirectional=False)
        self.bias = nn.Parameter(torch.Tensor(node_dim))
        self.bias.data.uniform_(-1.0 / math.sqrt(node_dim), 1.0 / math.sqrt(node_dim))


    def forward(self, node, edge_index, edge, hidden):
        num_node, node_dim = node.shape
        num_edge, edge_dim = edge.shape
        edge_index = edge_index.t().contiguous()

        #1. message :  m_j = SUM_i f(n_i, n_j, e_ij)  where i is neighbour(j)
        x_i     = torch.index_select(node, 0, edge_index[0])
        edge    = self.encoder(edge).view(-1,node_dim,node_dim)
        #message = x_i.view(-1,node_dim,1)*edge
        #message = message.sum(1)
        message = x_i.view(-1,1,node_dim)@edge
        message = message.view(-1,node_dim)
        message = scatter_('mean', message, edge_index[1], dim_size=num_node)
        message = F.relu(message +self.bias)

        #2. update: n_j = f(n_j, m_j)
        update = message

        #batch_first=True
        update, hidden = self.gru(update.view(1,-1,node_dim), hidden)
        update = update.view(-1,node_dim)

        return update, hidden

class Set2Set(torch.nn.Module):

    def softmax(self, x, index, num=None):
        x = x -  scatter_max(x, index, dim=0, dim_size=num)[0][index]
        x = x.exp()
        x = x / (scatter_add(x, index, dim=0, dim_size=num)[index] + 1e-16)
        return x

    def __init__(self, in_channel, processing_step=1):
        super(Set2Set, self).__init__()
        num_layer = 1
        out_channel = 2 * in_channel

        self.processing_step = processing_step
        self.in_channel  = in_channel
        self.out_channel = out_channel
        self.num_layer   = num_layer
        self.lstm = torch.nn.LSTM(out_channel, in_channel, num_layer)
        self.lstm.reset_parameters()

    def forward(self, x, batch_index):
        batch_size = batch_index.max().item() + 1

        h = (x.new_zeros((self.num_layer, batch_size, self.in_channel)),
             x.new_zeros((self.num_layer, batch_size, self.in_channel)))

        q_star = x.new_zeros(batch_size, self.out_channel)
        for i in range(self.processing_step):
            q, h = self.lstm(q_star.unsqueeze(0), h)
            q = q.view(batch_size, -1)

            e = (x * q[batch_index]).sum(dim=-1, keepdim=True) #shape = num_node x 1
            a = self.softmax(e, batch_index, num=batch_size)   #shape = num_node x 1
            r = scatter_add(a * x, batch_index, dim=0, dim_size=batch_size) #apply attention #shape = batch_size x ...
            q_star = torch.cat([q, r], dim=-1)

        return q_star

#message passing
class Net(torch.nn.Module):
    def __init__(self, node_dim=13, edge_dim=5, num_target=8):
        super(Net, self).__init__()
        self.num_propagate = 6
        self.num_s2s = 6

        self.preprocess = nn.Sequential(
            LinearBn(node_dim, 128),
            nn.ReLU(inplace=True),
            LinearBn(128, 128),
            nn.ReLU(inplace=True),
        )

        self.propagate = GraphConv(128, edge_dim)
        self.set2set = Set2Set(128, processing_step=self.num_s2s)


        #predict coupling constant
        self.predict = nn.Sequential(
            LinearBn(4*128, 1024),  #node_hidden_dim
            nn.ReLU(inplace=True),
            LinearBn( 1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_target),
        )

    def forward(self, node, edge, edge_index, node_index, coupling_index):

        num_node, node_dim = node.shape
        num_edge, edge_dim = edge.shape

        node   = self.preprocess(node)
        hidden = node.view(1,num_node,-1)

        for i in range(self.num_propagate):
            node, hidden =  self.propagate(node, edge_index, edge, hidden)

        pool = self.set2set(node, node_index)

        #---
        num_coupling = len(coupling_index)
        coupling_atom0_index, coupling_atom1_index, coupling_type_index, coupling_batch_index = \
            torch.split(coupling_index,1,dim=1)

        pool  = torch.index_select( pool, dim=0, index=coupling_batch_index.view(-1))
        node0 = torch.index_select( node, dim=0, index=coupling_atom0_index.view(-1))
        node1 = torch.index_select( node, dim=0, index=coupling_atom1_index.view(-1))

        predict = self.predict(torch.cat([pool,node0,node1],-1))
        predict = torch.gather(predict, 1, coupling_type_index).view(-1)
        return predict

In [23]:
def train2(net, schduler, optimizer, train_loader, valid_loader):
    
    start = timer()

    num_iters   = 3000  *1000
    iter_ = 0
    while  iter_<num_iters:
        
        train_loss = np.zeros(20,np.float32)
        batch_count = 0
        for node, edge, edge_index, node_index, coupling_value, coupling_index, infor in train_loader:

            # learning rate schduler -------------
            lr = schduler(iter_)
            if lr<0 : break
            adjust_learning_rate(optimizer, lr)
            rate = get_learning_rate(optimizer)

            # one iteration update  -------------
            #net.set_mode('train',is_freeze_bn=True)

            net.train()
            node = node.cuda()
            edge = edge.cuda()
            edge_index = edge_index.cuda()
            node_index = node_index.cuda()
            coupling_value = coupling_value.cuda()
            coupling_index = coupling_index.cuda()

            predict = net(node, edge, edge_index, node_index, coupling_index)

            optimizer.zero_grad()
            loss = criterion(predict, coupling_value)
            loss.backward()
            optimizer.step()
                
            # print statistics  ------------
            
            print(loss.shape)
            
            batch_loss[:1] = [loss.item()]
            train_loss += batch_loss
            batch_count += 1
            
            iter_+=1


        train_loss = train_loss / batch_count
        valid_loss = do_valid(net, valid_loader)
        print
        print('\r', end='', flush=True)
        print(time_to_str((timer() - start), 'min'), '|' , f'{iter_}/{num_iters}', '|', 'loss ', train_loss, '|', 'val_loss ', valid_loss, end='', flush=True)

In [ ]:

batch_size=16
node_dim = 93
edge_dim =  6
num_target = 8


cv = KFold(n_splits= 5, shuffle= True, random_state= 42)
splits = cv.split(molecule_names) 
for fold_n, (train_index, valid_index) in enumerate(splits):
    
    train_molecule_names = [molecule_names[i] for i in train_index]
    valid_molecule_names = [molecule_names[i] for i in valid_index]
    
    assert len(train_molecule_names) + len(valid_molecule_names) == len(molecule_names), 'len is not match'
    train_dataset = ChampsDataset(molecule_names=train_molecule_names, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    valid_dataset = ChampsDataset(molecule_names=valid_molecule_names, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, sampler=RandomSampler(valid_dataset), drop_last=True, num_workers=16, pin_memory=True, collate_fn=null_collate)
    
    net = Net(node_dim=node_dim,edge_dim=edge_dim, num_target=num_target).cuda()
    schduler = NullScheduler(lr=0.001)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=schduler(0))
    
    train2(net, schduler, optimizer, train_loader, valid_loader)
    
    break

In [9]:
atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

In [10]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
sub = pd.read_csv(f'{file_folder}/sample_submission.csv')
train['type0'] = train['type'].apply(lambda x: int(x[0]))
train['type1'] = train['type'].apply(lambda x: x[1:])

le = LabelEncoder()
le.fit(train.type1.tolist())
int_bond_type = le.transform(train.type1.tolist()) 
train['int_type1']= int_bond_type

test['type0'] = test['type'].apply(lambda x: int(x[0]))
test['type1'] = test['type'].apply(lambda x: x[1:])
int_bond_type = le.transform(test.type1.tolist()) 
test['int_type1']= int_bond_type

In [11]:
le.classes_

array(['JHC', 'JHH', 'JHN'], dtype='<U3')

In [12]:
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [13]:
x = magnetic_shielding_tensors.columns.values[2:]
x = magnetic_shielding_tensors[x].values
x = x.reshape(-1,3,3)
x = x + np.transpose(x,(0,2,1))
x = 0.5 * x
w, v = np.linalg.eigh(x)

sigma_iso = np.sum(w, axis=1)/3 
omega = w[:,2] - w[:,0]
kappa = 3 * (sigma_iso - w[:,1])/omega

magnetic_shielding_parameters = magnetic_shielding_tensors[magnetic_shielding_tensors.columns.values[:2]]
magnetic_shielding_parameters = pd.DataFrame(magnetic_shielding_parameters)
magnetic_shielding_parameters["sigma_iso"] = sigma_iso
magnetic_shielding_parameters["omega"] = omega
magnetic_shielding_parameters["kappa"] = kappa

In [14]:
atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

structures['atomic_radius'] = structures['atom'].apply(lambda x: atomic_radius[x])
structures['atomic_number'] = structures['atom'].apply(lambda x: atomic_number[x])
structures['atomic_mass'] = structures['atom'].apply(lambda x: atomic_mass[x])
structures['vanderwaalsradius'] = structures['atom'].apply(lambda x: vanderwaalsradius[x])
structures['covalenzradius'] = structures['atom'].apply(lambda x: covalenzradius[x])
structures['electronegativity'] = structures['atom'].apply(lambda x: electronegativity[x])
structures['ionization_energy'] = structures['atom'].apply(lambda x: ionization_energy[x])


In [15]:
structures.head()

,molecule_name,atom_index,atom,x,y,z,atomic_radius,atomic_number,atomic_mass,vanderwaalsradius,covalenzradius,electronegativity,ionization_energy
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,0.77,6,12.0107,185,77,2.55,11.2603
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,0.38,1,1.0079,120,30,2.20,13.5984
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,0.38,1,1.0079,120,30,2.20,13.5984
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,0.38,1,1.0079,120,30,2.20,13.5984
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,0.38,1,1.0079,120,30,2.20,13.5984


In [16]:
le = LabelEncoder()
le.fit(structures.atom.tolist())
int_atom = le.transform(structures.atom.tolist()) 
structures['int_atom'] = int_atom

In [17]:
le.classes_

array(['C', 'F', 'H', 'N', 'O'], dtype='<U1')

In [15]:
magnetic_shielding_parameters.head()

,molecule_name,atom_index,sigma_iso,omega,kappa
0,dsgdb9nsd_000001,0,195.316333,0.002708,0.032697
1,dsgdb9nsd_000001,1,31.460567,9.082838,0.999975
2,dsgdb9nsd_000001,2,31.460567,9.082793,0.999984
3,dsgdb9nsd_000001,3,31.460633,9.082804,0.999977
4,dsgdb9nsd_000001,4,31.460600,9.082853,0.999971


In [16]:
mulliken_charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [25]:
dipole_moments['rms'] = np.sqrt(dipole_moments['X']**2 + dipole_moments['Y']**2 + dipole_moments['Z']**2 )

In [18]:
dipole_moments.head()#MAE 3.x without scaled y, 2.x with scaled y

,molecule_name,X,Y,Z,rms
0,dsgdb9nsd_000001,0.0000,0.0,0.0000,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511,1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937,2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000,0.0000


In [19]:
potential_energy.head()#MAE 30.x with scaled y

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869


In [26]:
molecule_info = pd.merge(dipole_moments, potential_energy, how = 'left', left_on  = ['molecule_name'], right_on = ['molecule_name'])

In [27]:
molecule_info.head()

,molecule_name,X,Y,Z,rms,potential_energy
0,dsgdb9nsd_000001,0.0000,0.0,0.0000,0.0000,-40.52368
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256,1.6256,-56.56025
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511,1.8511,-76.42608
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937,2.8937,-93.42849
4,dsgdb9nsd_000007,0.0000,0.0,0.0000,0.0000,-79.83869


In [20]:
for col in molecule_info.columns.tolist():
    print(np.where(molecule_info[col].isna())[0].shape)

(0,)
(0,)
(0,)
(0,)
(0,)


In [21]:
atom_info = pd.merge(magnetic_shielding_parameters, mulliken_charges, how = 'left', left_on  = ['molecule_name','atom_index'], right_on = ['molecule_name','atom_index'])

In [22]:
atom_info.head()

,molecule_name,atom_index,sigma_iso,omega,kappa,mulliken_charge
0,dsgdb9nsd_000001,0,195.316333,0.002708,0.032697,-0.535689
1,dsgdb9nsd_000001,1,31.460567,9.082838,0.999975,0.133921
2,dsgdb9nsd_000001,2,31.460567,9.082793,0.999984,0.133922
3,dsgdb9nsd_000001,3,31.460633,9.082804,0.999977,0.133923
4,dsgdb9nsd_000001,4,31.460600,9.082853,0.999971,0.133923


In [23]:
for col in atom_info.columns.tolist():
    print(np.where(atom_info[col].isna())[0].shape)

(0,)
(0,)
(0,)
(0,)
(0,)
(0,)


In [28]:
y_std = np.std(molecule_info[['potential_energy','rms']].values, axis=0)
y_mean = np.mean(molecule_info[['potential_energy','rms']].values, axis=0)
y_std, y_mean

(array([39.82863665,  1.49279267]), array([-410.87405548,    2.67236092]))

In [29]:
def _worker(item, df_bonds, df_structures, molecule_info, has_y):
    
    idx = item[0]
    molecule_name = item[1]
    
    # point attribute
    x = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['int_atom','atomic_radius', 'atomic_number', 'atomic_mass', 'vanderwaalsradius', 'covalenzradius', 'electronegativity', 'ionization_energy']].values, dtype=torch.float)
    # position
    pos = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['x','y','z']].values, dtype=torch.float)

    edge_index = []
    edge_attr = []

    df_molecule_i = df_bonds[df_bonds['molecule_name']==molecule_name]
    molecule_info_i = molecule_info[molecule_info['molecule_name']==molecule_name]

    if has_y:
        y = (molecule_info_i[['potential_energy','rms']].values-y_mean)/y_std
        y = torch.tensor(y, dtype=torch.float)

    for idx, row in df_molecule_i.iterrows():
        edge_index.append([row['atom_index_0'], row['atom_index_1']])
        edge_index.append([row['atom_index_1'], row['atom_index_0']])
        edge_attr.append([row['type0'], row['int_type1']])
        edge_attr.append([row['type0'], row['int_type1']])

    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    if has_y:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr, y=y)
    else:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr)
    d_ =  {'idx':idx, 'molecule_name':molecule_name, 'data':data}
#     print(d_)
    return d_


def generate_datalist(df_bonds, df_structures, molecule_info, molecule_names, has_y = True):
    N_ = len(molecule_names)
    segments = N_//100
    data_list = []
    
#     data_list = Parallel(_worker,{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y}).run(zip(np.arange(N_), molecule_names), n_jobs=4)
    
#     for start in tqdm_notebook(range(0, N_, segments)):
#         end = start + segments if start + segments < N_ else N_
#         res = Parallel(_worker,{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y}).run(zip(np.arange(start,end,1), molecule_names[start:end]), n_jobs=4)
    for item in tqdm_notebook(enumerate(molecule_names), total=N_):
        d_ = _worker(item, **{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y})
        data_list.append(d_)
    df_ = pd.DataFrame(data_list)
    df_ = df_.sort_values(by=['idx']).reset_index(drop=True)
    return df_
    

In [ ]:
molecule_names = train.molecule_name.unique().tolist()

In [36]:
df_train = generate_datalist(train, structures, molecule_info, molecule_names=molecule_names)

In [31]:
molecule_names = test.molecule_name.unique().tolist()

In [ ]:
test = generate_datalist(test, structures, molecule_info, molecule_names=molecule_names)

In [37]:
# df_train.to_pickle('../../data/feature/graph_train.gzip', compression='gzip')

In [5]:
df_train = pd.read_pickle('../../data/feature/graph_train.gzip', compression='gzip')

In [6]:
df_train.head()

,data,idx,molecule_name
0,"[(edge_attr, [tensor([1., 0.]), tensor([1., 0....",9,dsgdb9nsd_000001
1,"[(edge_attr, [tensor([1., 2.]), tensor([1., 2....",15,dsgdb9nsd_000002
2,"[(edge_attr, [tensor([2., 1.]), tensor([2., 1....",16,dsgdb9nsd_000003
3,"[(edge_attr, [tensor([1., 0.]), tensor([1., 0....",18,dsgdb9nsd_000005
4,"[(edge_attr, [tensor([1., 0.]), tensor([1., 0....",45,dsgdb9nsd_000007


In [7]:
df_train.data.values[0]

Data(edge_attr=[20, 2], edge_index=[2, 20], pos=[5, 3], x=[5, 8], y=[1, 2])

In [8]:
def generate_dataLoader(df, molecule_names, shuffle=True, batch_size=32, **kwargs):
    data_list = df[df['molecule_name'].isin(molecule_names)]['data'].tolist()
    return DataLoader(data_list, batch_size=batch_size, shuffle=shuffle, **kwargs)

In [40]:
for i in train_molecule_names
train_molecule_names, valid_molecule_names =  train_test_split(molecule_names, train_size=.8, shuffle=True)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [42]:
# test_loader = generate_dataLoader(df_data_train, test_molecule_names)
val_loader = generate_dataLoader(df_train, valid_molecule_names)
train_loader = generate_dataLoader(df_train, train_molecule_names)

In [43]:
gnn_param = {
    'node_expand_dim':64,
    'link_expand_dim':128,
    'node_dim':8,
    'link_dim':2,
    'conv_layers':3
}
# node_expand_dim = 64
# link_expand_dim = 128
# node_dim = 8
# link_dim = 2
# conv_layers = 3

class Net(torch.nn.Module):
    
    def __init__(self, node_expand_dim, link_expand_dim, node_dim, link_dim, conv_layers):
        super(Net, self).__init__()
        
        self.node_expand_dim = node_expand_dim
        self.link_expand_dim = link_expand_dim
        self.node_dim = node_dim
        self.link_dim = link_dim
        self.conv_layers = conv_layers
        
        self.lin0 = torch.nn.Linear(node_dim, node_expand_dim)
        nn = Sequential(Linear(link_dim, link_expand_dim), ReLU(), Linear(link_expand_dim, node_expand_dim * node_expand_dim))
        self.conv = NNConv(node_expand_dim, node_expand_dim, nn, aggr='mean', root_weight=False)
        self.gru = GRU(node_expand_dim, node_expand_dim)
        self.set2set = Set2Set(node_expand_dim, processing_steps=3)
        self.lin3 = torch.nn.Linear(node_expand_dim, 2 * node_expand_dim)
        self.lin1 = torch.nn.Linear(2 * node_expand_dim, node_expand_dim)
        self.lin2 = torch.nn.Linear(node_expand_dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)
        for i in range(self.conv_layers):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
            
        out = self.set2set(out, data.batch)
#         out = F.relu(self.lin3(out))
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
#         print(out.shape)
#         out = out.view(-1)
        return out



class GNN(object):
    
    def __init__(self, node_expand_dim, link_expand_dim, node_dim, link_dim, conv_layers):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = Net(node_expand_dim, link_expand_dim, node_dim, link_dim, conv_layers).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min', factor=0.7, patience=5, min_lr=0.00001)
        return

    def fit(self, train_loader, val_loader, epochs, y_std, y_mean, y_index=0, verbose=0):

        his = []
        for epoch in range(1, epochs+1):
            lr = self.scheduler.optimizer.param_groups[0]['lr']
            loss = self._train(epoch, train_loader, y_index, self.model, self.device, self.optimizer)
            val_error = self._test(val_loader, y_index, self.model, self.device, self.optimizer, y_std, y_mean)
            self.scheduler.step(val_error)
            his.append({'epoch':epoch, 'lr':lr, 'loss':loss, 'val_error':val_error})
            if verbose > 0:
#                 if epoch % (epochs//10) == 0:
                print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}'.format(epoch, lr, loss, val_error))
        return his
    
    def predict(self, loader, y_std, y_mean, y_index=0):
        model.eval()
        y_pred_list  = []
        for data in loader:
            data = data.to(device)
            std = torch.tensor(y_std[y_index:y_index+1].reshape(1,1), dtype=torch.float)
            std = std.to(device)
            y_pred = model(data)
            y_pred_list.append(y_pred.item())
        y_pred = np.array(y_pred_list)
        y_pred = y_pred*y_std + y_mean
        return y_pred
    
    def _train(self, epoch, loader, y_index, model, device, optimizer):
        model.train()
        loss_all = 0
        for data in loader:
            data = data.to(device)
            optimizer.zero_grad()
            y_pred = model(data)
            loss = F.mse_loss(y_pred, data.y[:,y_index:y_index+1])
            loss.backward()
            loss_all += loss.item() * data.num_graphs
            optimizer.step()
        return loss_all / len(loader.dataset)


    def _test(self, loader, y_index, model, device, optimizer, y_std, y_mean):
        model.eval()
        error = 0
        for data in loader:
            data = data.to(device)
            std = torch.tensor(y_std[y_index:y_index+1].reshape(1,1), dtype=torch.float)
            std = std.to(device)
            y_pred = model(data)
#             print(y_pred.shape)            
            error += (y_pred.mm(std) - data.y[:,y_index:y_index+1].mm(std)).abs().sum().item()  # MAE
        return error / len(loader.dataset)

In [44]:
gnn = GNN(**gnn_param)

In [ ]:
his  = gnn.fit(train_loader, val_loader, 100, y_std, y_mean, y_index=1, verbose=1)

Epoch: 001, LR: 0.001000, Loss: 0.8995454, Validation MAE: 1.0764222
Epoch: 002, LR: 0.001000, Loss: 0.8312789, Validation MAE: 0.9917343
Epoch: 003, LR: 0.001000, Loss: 0.8373433, Validation MAE: 1.0545565
Epoch: 004, LR: 0.001000, Loss: 0.8354087, Validation MAE: 1.0621529
Epoch: 005, LR: 0.001000, Loss: 0.8302558, Validation MAE: 0.9906469
Epoch: 006, LR: 0.001000, Loss: 0.7646985, Validation MAE: 0.9733653
Epoch: 007, LR: 0.001000, Loss: 0.7212076, Validation MAE: 0.9276138
Epoch: 008, LR: 0.001000, Loss: 0.7138798, Validation MAE: 0.9443923
Epoch: 009, LR: 0.001000, Loss: 0.7097099, Validation MAE: 0.9370616
Epoch: 010, LR: 0.001000, Loss: 0.7077996, Validation MAE: 0.9285092
Epoch: 011, LR: 0.001000, Loss: 0.7070415, Validation MAE: 0.9407990
Epoch: 012, LR: 0.001000, Loss: 0.7046720, Validation MAE: 0.9240429
Epoch: 013, LR: 0.001000, Loss: 0.7047568, Validation MAE: 0.9255332
Epoch: 014, LR: 0.001000, Loss: 0.7055413, Validation MAE: 0.9225434
Epoch: 015, LR: 0.001000, Loss: 0.

In [175]:
y_pred  = gnn.predict(val_loader, y_std, y_mean, y_index=1)

TypeError: predict() missing 2 required positional arguments: 'y_std' and 'y_mean'